In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import keras
import tensorflow as tf
import nltk
import re
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from nltk.corpus import stopwords
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from tensorflow.keras.preprocessing.text import one_hot
from nltk.stem.porter import PorterStemmer
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
true=pd.read_csv('../input/fake-and-real-news-dataset/True.csv')
fake=pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')

# EDA

In [ ]:
true.head(10)

In [ ]:
fake.head(10)

In [ ]:
true.shape

In [ ]:
fake.shape

In [ ]:
true.isnull().sum()

In [ ]:
fake.isnull().sum()

In [ ]:
fake.info()

In [ ]:
true.info()

In [ ]:
true['target']=1
fake['target']=0

In [ ]:
news_data=pd.concat([true,fake],ignore_index=True,sort=False)

In [ ]:
news_data.head(5)

In [ ]:
news_data.shape

In [ ]:
plt.figure(figsize=(5,5))
sns.set(style='darkgrid')
sns.countplot(news_data.target)
plt.title('Data distribution of fake and real data')

In [ ]:
sns.set(style='darkgrid')
plt.figure(figsize=(15,5))
plt.title("Data distribution of fake and real data")
sns.countplot(x='subject',data=news_data,hue='target')
plt.xlabel('news')
plt.ylabel('total no count')

In [ ]:
news_data['text']=news_data['title']+''+news_data['text']

In [ ]:
data=news_data[['text','target']]

# Tokenization:-
Tokenization is task splitting text into meaningfull segment called tokens.the input to tokenizer is a unicode text and the output is doc object

In [ ]:
spacy_tok = spacy.load('en_core_web_sm')
sample_data=data.text[50]
sample_data

In [ ]:
parsed_data=spacy_tok(sample_data)
parsed_data

# #Here we visualize the data differnent method

In [ ]:
!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py

In [ ]:
import explacy
explacy.print_parse_info(spacy_tok, 'Trump urges Congress to pass short-term spending billWASHINGTON (Reuters)')

In [ ]:
 explacy.print_parse_info(spacy_tok,data.text[1])

After tokenization we can parse tag variety of part of speech to paragraph text.Spacy uses stats models in background to predict which tag will go each word based on the context
Lemmatization:-it is the process of extracting form of the word

In [ ]:
tokenize=pd.DataFrame()
for i,token in enumerate(parsed_data):
    tokenize.loc[i,'text']=token.text
    tokenize.loc[i,'lemma']=token.lemma_
    tokenize.loc[i,'pos']=token.pos_
    tokenize.loc[i,'dep']=token.dep_
    tokenize.loc[i, 'is_punctuation'] = token.is_punct
    tokenize.loc[i, 'shape'] = token.shape_
    tokenize.loc[i, 'is_alpha'] = token.is_alpha
    tokenize.loc[i, 'is_stop'] = token.is_stop
tokenize[:5]

visulization through displacy

In [ ]:
spacy.displacy.render(parsed_data, style='ent')

In [ ]:
spacy.explain('GPE') # to explain POS tag

Dependency parsing
Syntactic Parsing or Dependency Parsing is process of identifyig sentenses and assigning a syntactic structure to it. As in Subject combined with object makes a sentence. Spacy provides parse tree which can be used to generate this structure.
Sentense Boundry Detection

In [ ]:
sentence_spans = list(parsed_data.sents)
sentence_spans

In [ ]:
from spacy import displacy
displacy.render(parsed_data, style='dep', jupyter=True,options={'distance': 140})

In [ ]:
options = {'compact': True, 'bg': 'violet','distance': 140,
           'color': 'white', 'font': 'Trebuchet MS'}
displacy.render(parsed_data,jupyter=True, style='dep', options=options)

In [ ]:
data=data[:1000]

In [ ]:
## Get the Independent Features
X=data['text']
y=data['target']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
#here we take vocabulry size
voc_size=5000

In [ ]:
nltk.download('stopwords')

In [ ]:
X.shape

In [ ]:
ps=PorterStemmer()
corpus=[]
for i in range(0,len(X)):
    print(i)
    news=re.sub('[^a-zA-Z]',' ',data['text'][i])
    news=news.lower()
    news=news.split()
    news=[ps.stem(word) for word in news if not word in stopwords.words('english')]
    news=' '.join(news)
    corpus.append(news)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
#Embedding the text data
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[0]

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(1000))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape

In [ ]:
y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final, test_size=0.20, random_state=0)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
## Creating bidirectional 
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.4))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred1=model1.predict_classes(X_test)

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))